# 13주
A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.



함수 | 설명

-----|-----

setA | 직접 입력하지 않고, 가위, 바위, 보를 무작위로 생성하여 내고 ```1000 wei```를 베팅한다.

setB | 게임 플레이어가 직접 입력하고, ```1000 wei```를 베팅한다.

play | setA, setB 입력이 끝나고 컴퓨터가 실행하는 것으로 하고, 승패가 결정짓는다.

distributeBetAmount | 승패에 따라 분배한다. 승자가 패자의 내기 금액 ```1000 wei```를 가지게 된다.

getMatchResult() | 누가 이겼는지 A, B 승자를 포함한 문자열 "A wins", "B wins", "tie" 결과를 출력한다.



노드에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력 (컨트랙 상의 잔고를 말한다)

- setA() 실행. 플레이어 A는 컴퓨터가 대행하는 것으로 하고, 내기금액 걸고 가위바위보 중 하나를 무작위로 선택

- setB() 실행. B는 자신이 직접 내기금액 걸고, 가위바위보 중 하나를 선택하여 입력

- play() 실행해서, 승부를 결정

- getMatchResult() 실행해서, 승부의 결과를 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력 (컨트랙 상의 잔고를 말한다)

## 함수생성

In [25]:
%%writefile src/Rsp.sol
pragma solidity ^0.6.4;

contract Rsp {
    struct Player {
        string name;
        uint betting;
        uint rsp;
    }

    int256 result = -1;
    address payable owner;
    address addrA;
    address addrB;
    uint balance;

    constructor() public {
        owner = msg.sender;
    }

    mapping(address => Player) playerMap;
    mapping(string => address) addressByName;
    mapping(address => uint) balanceOf;
    
    function getBalance() public view returns(uint, uint, uint) {
        return(balanceOf[addrA], balanceOf[addrB], address(this).balance);
    }

    function setAddressAB(address receiver) public {
        addrA = address(this);
        addrB = receiver;
    }

    function deposit(uint amount) payable public onlyOwner {
        require(msg.value == amount);
        balanceOf[addrA] += amount;
        balanceOf[addrB] += amount;
    }

    function setA() public {
        uint _rsp= uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        Player memory m = Player("player_A", 1000, _rsp);
        playerMap[addrA] = m;
        addressByName["player_A"] = addrA;
    }

    function setB(uint _betting, uint _rsp) public {
        Player memory m2 = Player("player_B", _betting, _rsp);
        playerMap[addrB] = m2;
        addressByName["player_B"] = addrB;
    }

    function play() public {
        // 0 : rock, 1 : scissor, 2 : paper
        Player memory u1 = playerMap[addressByName["player_A"]];
        Player memory u2 = playerMap[addressByName["player_B"]];
        uint a = u1.rsp;
        uint b = u2.rsp;

        if(a == b) {
            result = 0; // tie
        }
        else if((a==0 && b==1) || (a==1 && b==2) || (a==2 && b==0)) {
            result = 1; // a wins
        }
        else {
            result = 2; // b wins
        }
    }

    function distributeBetAmount() public payable {
        Player memory u1 = playerMap[addressByName["player_A"]];
        Player memory u2 = playerMap[addressByName["player_B"]];
        uint a_betting = u1.betting;
        uint b_betting = u2.betting;

        if(result == 1) { // a wins
            balanceOf[addressByName["player_A"]] += b_betting;
            balanceOf[addressByName["player_B"]] -= b_betting;
        }
        else if(result == 2) { // b wins
            balanceOf[addressByName["player_A"]] -= a_betting;
            balanceOf[addressByName["player_B"]] += a_betting;
        }
    }

    function getMatchResult() public view returns(string memory) {
        if(result == 1) {
            return("A wins");
        }
        else if(result == 2) {
            return("B wins");
        }
        else {
            return("tie");
        }
    }

    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/Rsp.sol


In [26]:
!solc src/Rsp.sol --combined-json abi,bin > src/Rsp.json

## 배포

In [27]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Rsp.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract form " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 1255242, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        }})
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/RspDeploy.js


In [28]:
!node src/RspDeploy.js

- contract name :  [ 'src/Rsp.sol:Rsp' ]
Deploying the contract form 0xf0B05982bB5D9E5e85b2382075B18662A2570e16
---> The contract deployed to : 0x12964CD93C8d6DF3a30dcBD246499C84e00154d9


## 사용

In [31]:
%%writefile src/RspUSe.js
var Web3 = require('web3');
var fs = require('fs');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiBinJson = require('./Rsp.json');

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray = JSON.parse(_abiBinJson.contracts[contractName].abi);
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var rsp = new web3.eth.Contract(_abiArray, '0x12964CD93C8d6DF3a30dcBD246499C84e00154d9');
    const accounts = await web3.eth.getAccounts();
    console.log("- account : " + accounts[0]);
    await rsp.methods.setAddressAB(accounts[1]).send({from: accounts[0], gas: 122209});
    await rsp.methods.deposit(10000).send({from: accounts[0], gas: 122209, value: 10000});
    rsp.methods.getBalance().call()
        .then(function(value) {
            console.log(" - init balance \n A: ", value[0], " B : ", value[1], " Contract : ", value[2]);
        })
    await rsp.methods.setA().send({from: accounts[0], gas: 122209});
    await rsp.methods.setB(1000, 2).send({from: accounts[0], gas: 122209});
    await rsp.methods.play().send({from: accounts[0], gas: 122209});
    await rsp.methods.distributeBetAmount().send({from: accounts[0], gas: 122209});
    rsp.methods.getMatchResult().call().then(console.log);
    rsp.methods.getBalance().call()
        .then(function(value) {
            console.log(" - after game \n A: ", value[0], " B : ", value[1], " Contract : ", value[2]);
        });
}

doIt();

Overwriting src/RspUSe.js


In [34]:
!node src/RspUse.js

- contract name :  [ 'src/Rsp.sol:Rsp' ]
- account : 0xf0B05982bB5D9E5e85b2382075B18662A2570e16
 - init balance 
 A:  32000  B :  28000  Contract :  40000
B wins
 - after game 
 A:  31000  B :  29000  Contract :  40000
